In [2]:
import numpy as np
import matplotlib.pyplot as plt
import json
from google.protobuf.json_format import MessageToDict
from helpers.read import Db

db_name = 'fixed_n_high_precision'
db  = Db('./' + db_name + '.db')
gen = db.rows()
target_distance = 30

def format_layer(layer):
    return MessageToDict(layer)    

def format_doc(reflectance, layers):
    return {
        "layers": [format_layer(layer) for layer in layers],
        "reflectance": reflectance
    }


docs = []

for sim in gen:
    ref = np.asarray(sim.reflectance)
    abs = np.asarray(sim.absorption).reshape((100, 100))

    radial_extent = sim.details.radialExtent
    bin_size = (radial_extent.max - radial_extent.min) / 100
    i = np.floor(target_distance / bin_size).astype(int)

    docs.append(format_doc(ref[i], sim.details.layers))

with open('./results/' + db_name + str(target_distance) + '.json', 'w', encoding='utf-8') as f:
    json.dump(docs, f, ensure_ascii=False, indent=4)
